In [1]:
import numpy as np
import pandas as pd

# 一、string类型的性质
1. string与object的区别
string类型和object不同之处有三：
① 字符存取方法（string accessor methods，如str.count）会返回相应数据的Nullable类型，而object会随缺失值的存在而改变返回类型
② 某些Series方法不能在string上使用，例如： Series.str.decode()，因为存储的是字符串而不是字节
③ string类型在缺失值存储或运算时，类型会广播为pd.NA，而不是浮点型np.nan
其余全部内容在当前版本下完全一致，但迎合Pandas的发展模式，我们仍然全部用string来操作字符串
2. string类型的转换
如果将一个其他类型的容器直接转换string类型可能会出错：

In [2]:
pd.Series([1,'1.']).astype('str').astype('string')

0     1
1    1.
dtype: string

In [5]:
pd.Series([1,2]).astype('str').astype('string')

0    1
1    2
dtype: string

# 二、拆分与拼接
1. str.split方法

In [6]:
s = pd.Series(['a_b_c', 'c_d_e', np.nan, 'f_g_h'], dtype="string")
s

0    a_b_c
1    c_d_e
2     <NA>
3    f_g_h
dtype: string

In [7]:
s2 = pd.Series(list('abc'),index=[1,2,3],dtype='string')
s2

1    a
2    b
3    c
dtype: string

# 三、替换
广义上的替换，就是指str.replace函数的应用，fillna是针对缺失值的替换，上一章已经提及
提到替换，就不可避免地接触到正则表达式，这里默认读者已掌握常见正则表达式知识点，若对其还不了解的，可以通过这份资料来熟悉
1. str.replace的常见用法

In [8]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca','', np.nan, 'CABA', 'dog', 'cat'],dtype="string")
s

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6    <NA>
7    CABA
8     dog
9     cat
dtype: string

In [9]:
s.str.replace(r'^[AB]','123456')

0       123456
1       123456
2            C
3    123456aba
4    123456aca
5             
6         <NA>
7         CABA
8          dog
9          cat
dtype: string

### 2. 子组与函数替换
通过正整数调用子组（0返回字符本身，从1开始才是子组）

In [10]:
s.str.replace(r'([ABC])(\w+)',lambda x:x.group(2)[1:]+'*')

0       A
1       B
2       C
3     ba*
4     ca*
5        
6    <NA>
7     BA*
8     dog
9     cat
dtype: string

# 3. 关于str.replace的注意事项
首先，要明确str.replace和replace并不是一个东西：
str.replace针对的是object类型或string类型，默认是以正则表达式为操作，目前暂时不支持DataFrame上使用
replace针对的是任意类型的序列或数据框，如果要以正则表达式替换，需要设置regex=True，该方法通过字典可支持多列替换
但现在由于string类型的初步引入，用法上出现了一些问题，这些issue有望在以后的版本中修复
（a）str.replace赋值参数不得为pd.NA
这听上去非常不合理，例如对满足某些正则条件的字符串替换为缺失值，直接更改为缺失值在当下版本就会报错

In [11]:
pd.Series(['A','B'],dtype='string').astype('O').replace(r'[A]',pd.NA,regex=True).astype('string')

0    <NA>
1       B
dtype: string

In [12]:
pd.Series(['A',np.nan],dtype='string').str.replace('A','B')

0       B
1    <NA>
dtype: string